In [ ]:
import os
import json
import time
import csv
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from dotenv import load_dotenv

In [ ]:
load_dotenv(verbose=True)
driver = webdriver.Chrome('./chromedriver')
tj_base = 'http://m.tjmedia.co.kr/tjsong/song_search_result.asp?strType=1&strText='
tj_query = '&strCond=0&strSize01=100'
movie_url = os.getenv('SEARCH_BASE')

In [ ]:
data_list = []

with open('artist_title_list.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        data_list.append([line[0].replace('\xa0', ' '), line[1].replace('\xa0', ' ')])
data_list.pop(0)

data_list

## get karaoke data

In [ ]:
insertion_music_list = []

count = 0;
true_count = 0;

for data in data_list:
    try:
        find = False
        title = data[1].split('(')[0].replace(' ', '').lower()
        artist = data[0]

        driver.get(tj_base + title + tj_query)

        result_list = driver.find_elements_by_css_selector('#BoardType1 tbody')[0]
        result_list = result_list.find_elements_by_css_selector('tr')
        result_list.pop(0)

        ### result_td
        ### 0: karaoke number
        ### 1: music title
        ### 2: artist name
        artist_temp = artist.replace(' ', '').lower()
        for result in result_list:
            result_td = result.find_elements_by_css_selector('td')   
            title_search = result_td[1]

            ## MR 제거
            if len(title_search.find_elements_by_css_selector('*')) > 2:
                continue
            else:
                artist_search = result_td[2].text.replace(' ', '').lower()
                if artist_search in artist_temp or artist_temp in artist_search:
                    find = True
                    true_count += 1
                    insertion_music_list.append([result_td[0].text, result_td[1].text, result_td[2].text])
                    break
                else:
                    continue
    except:
        find=False
    count += 1

insertion_music_list

In [ ]:
error = 0;
patch = 0;
post = 0;

for insertion_music in insertion_music_list:
    datas = {
        'chart': -1,
        'title': insertion_music[1],
        'artist': insertion_music[2],
        'number': insertion_music[0]
    }
    url = 'http://localhost:3000/dev/music/chart'
    res = requests.post(url, json=datas)
    if(res.json()['status'] == False):
        error = error + 1
    else:
        if(res.json()['body'] == 'patch'):
            patch = patch + 1
        if(res.json()['body'] == 'post'):
            post = post + 1
            
print(error, patch, post)

In [ ]:
res = requests.get('http://localhost:3000/dev/music/youtube')
empty_music_list = res.json()['body']
request_body_list = []

for empty_music in empty_music_list:
    count = 0;
    while(True):
        try:
            driver.get(movie_url + empty_music['artist'] + '+' + empty_music['title'])
            code = driver.find_elements_by_css_selector('a#video-title')[0].get_attribute('href')
            request_body_list.append([empty_music['id'], code.split('watch?v=')[1]])
            break
        except:
            if(count > 10):
                break
            count += 1
            continue
            
request_body_list

In [ ]:
success = 0
error = 0
for request_body in request_body_list:
    datas = {
        'id': request_body[0],
        'code': request_body[1]
    }
    url = 'http://localhost:3000/dev/music/youtube'
    res = requests.patch(url, json=datas)
    if(res.json()['status'] == False):
        error = error + 1
    else:
        success = success + 1
        
print(error, success)